# Disaggregation of appliances from generated load profiles

In [1]:
# General libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.optimizers import Adam

%matplotlib inline

In [4]:
df_import = pd.read_excel("data/DeviceProfiles.HH1.Electricity.xlsx")
df_import.head()

,Electricity.Timestep,Time,"HH1 - Living room - Living Room Light (Energy Saving Lamp, 20W) [kWh]",HH1 - Kitchen - Kitchen Light (20W) [kWh],HH1 - Bath - Bathroom Light (20W) [kWh],HH1 - Bath - Bathroom Mirror Light 10 W (LED) [kWh],HH1 - Bedroom - Bedroom Light (20W) [kWh],HH1 - Children's room - Children Room Light Device (20W) [kWh],HH1 - Kitchen - AFK BM-2N [kWh],HH1 - Kitchen - Egg Cooker / Russell Hobbs 14048-56 Stylo [kWh],...,HH1 - Living room - TV Medion MD20123_DE_A [kWh].1,HH1 - Living room - CD/DVD Player / Philips DVDR 725 H [kWh].1,HH1 - Living room - Home Server 50 W [kWh],HH1 - Bath - Electric Razor Braun Cruzer 5 [kWh],HH1 - Living room - Laptop Sony Vaio SVE151G11M [kWh].1,HH1 - Living room - Router O2 Box 6431 [kWh],HH1 - Children's room - TV Medion MD20123_DE_A [kWh].1,HH1 - Children's room - Sony PlayStation 3 [kWh].1,HH1 - Kitchen - Vacuum Cleaner Robot / iRobot Roomba 555 [kWh].1,"HH1 - Vehicle Pool - Car 2, 22kW Charging Power, avg. Speed 30 km/h [kWh]"
0,0,2021-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.488916e-08,2.700302e-07,0.000015,2.720909e-07,6.611897e-07,0.000002,7.539537e-08,1.396108e-06,6.814155e-07,0
1,1,2021-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.431976e-08,2.600469e-07,0.000014,2.720909e-07,7.880510e-07,0.000002,6.862741e-08,1.477735e-06,8.728055e-07,0
2,2,2021-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.664294e-08,2.971025e-07,0.000014,2.720909e-07,8.491777e-07,0.000002,6.825719e-08,1.381125e-06,6.948173e-07,0
3,3,2021-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.614913e-08,2.631576e-07,0.000014,2.720909e-07,8.335628e-07,0.000002,8.165245e-08,1.162740e-06,8.729020e-07,0
4,4,2021-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.619572e-08,3.021713e-07,0.000013,2.720909e-07,7.016996e-07,0.000002,6.404121e-08,9.997714e-07,8.563752e-07,0


In [9]:
df_selected = df_import[["Electricity.Timestep",
    "Time",
    "HH1 - Kitchen - Single Stove Plate [kWh]",
    "HH1 - Kitchen - Nespresso Coffee Machine, Single Cup [kWh]",
    "HH1 - Kitchen - Microwave / Panasonic NN 5259 [kWh]",
    "HH1 - Kitchen - Dishwasher NEFF SD6P1F (2011) [kWh]",
    "HH1 - Kitchen - Washing Machine / Bosch WAE 28143 [kWh]"]].copy()
#del(df_import)

In [20]:
def count_non_zero_rows(dataframe, column):
    return len(dataframe) - dataframe[column].isin([0]).sum()

In [24]:
count_non_zero_rows(df_selected, "HH1 - Kitchen - Washing Machine / Bosch WAE 28143 [kWh]")

6692

In [25]:
name_mappings = {
    "HH1 - Kitchen - Single Stove Plate [kWh]" : "Stove",
    "HH1 - Kitchen - Nespresso Coffee Machine, Single Cup [kWh]" : "Coffee_machine",
    "HH1 - Kitchen - Microwave / Panasonic NN 5259 [kWh]" : "Microwave",
    "HH1 - Kitchen - Dishwasher NEFF SD6P1F (2011) [kWh]" : "Dishwasher",
    "HH1 - Kitchen - Washing Machine / Bosch WAE 28143 [kWh]" : "Washing_machine"
}

In [82]:
df = df_selected.rename(columns=name_mappings).copy()
#del(df_selected)

In [76]:
df.head(61)

,Electricity.Timestep,Time,Stove,Coffee_machine,Microwave,Dishwasher,Washing_machine
0,0,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
1,1,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
2,2,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
3,3,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
4,4,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
56,56,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
57,57,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
58,58,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
59,59,2021-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0


## Create target column

In [83]:
df["Target"] = df[list(name_mappings.values())].sum(axis=1)

## Fix timestamp seconds

In [99]:
df["Time"] = [dt.replace(second=(df["Electricity.Timestep"][i]%60)) for i,dt in enumerate(df["Time"])]

## Export data for further preprocessing

In [103]:
df.to_csv("data/data_reduced.csv")